In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
from datetime import datetime, timedelta
from utils import format_number

from config import LAST_N_DAYS, INCENTIVE_PROGRAMS_ONLY, COL_NAMES_TO_INCLUDE, GOV_INCENTIVE_PROGRAMS_ONLY

pd.options.display.float_format = "{:,.2f}".format

In [2]:
def extract_source(source_string):
    source_list = source_string.split("-")
    if len(source_list) > 1:
        return source_list[1].strip()  # strip() is used to remove any leading/trailing spaces
    else:
        return source_string.strip()
    
def cleanup_string(source_string):
    return source_string.replace(" ", "").lower()

# Incentive Program Summary
Status of programs live, completed and to be announced by season.

In [3]:
df_info = pd.read_csv("inputs/" + "op_incentive_program_info" + ".csv")

if INCENTIVE_PROGRAMS_ONLY:
    df_info = df_info[df_info["Incentive / Growth Program Included?"] == "Yes"]

if GOV_INCENTIVE_PROGRAMS_ONLY:
    df_info = df_info[(df_info["Incentive / Growth Program Included?"] == "Yes") & (df_info["Source"] != "Partner Fund")]

In [4]:
# convert to datetime
df_info["announced_date"] = pd.to_datetime( df_info["Announced On"] )
df_info["End Date"] = pd.to_datetime( df_info["End Date"] )

# convert program status into ordered categorical type
cat_size_order = CategoricalDtype(
    ["Live ‎🔥", "Coming soon ‎⏳", "Completed"], 
    ordered=True
)
df_info["Status"] = df_info["Status"].astype(cat_size_order)


# clean up for columns needed
df_info = df_info[["Source","Status","# OP Allocated","App Name","announced_date", "End Date", "App Name Map Override"]]

summary = pd.pivot_table(df_info, values=["# OP Allocated", "App Name"], index=["Status", "Source"], \
    aggfunc={"# OP Allocated":"sum", "App Name":"count"})

# calculate subtotals on program status
result=pd.concat([summary,summary.groupby(level=0).sum().assign(item_name="Sub Total").set_index("item_name",append=True)]).sort_index(level=[0,1])
result = result.sort_index(level=[0, 1], ascending=[True, False])

# add grand total to summary
result.loc[("Grand Total"), "# OP Allocated"] = summary["# OP Allocated"].sum()
result.loc[("Grand Total"), "App Name"] = summary["App Name"].sum()

# cleanup display
result["# Programs"] = result["App Name"].astype(int)
result["# OP Allocated (M)"] = result["# OP Allocated"].apply(format_number)

# calculate percentage of total
result.loc[(slice(None), "Sub Total"), "# OP Allocated"] / summary["# OP Allocated"].sum()
result["% OP Allocated"] = round(result.loc[(slice(None), "Sub Total"), "# OP Allocated"] / summary["# OP Allocated"].sum() * 100).\
    astype(str).replace("\.0", "", regex=True) + "%"
result["% OP Allocated"].fillna("-", inplace=True)

result = result.replace((0, "0.0M"), "-")
display(result.drop(columns=["# OP Allocated", "App Name"]))


# Programs # OP Allocated (M)  \
Status         Source                                                
Live ‎🔥        Sub Total                     34              41.7M   
               Governance - Season 3          -                0.0   
               Governance - Season 2          9               6.5M   
               Governance - Season 1          9               4.5M   
               Governance - Phase 0          16              30.7M   
Coming soon ‎⏳ Sub Total                     34              11.7M   
               Governance - Season 3         12               2.1M   
               Governance - Season 2         15               6.1M   
               Governance - Season 1          4               1.3M   
               Governance - Phase 0           3               2.2M   
Completed      Sub Total                      2               1.2M   
               Governance - Season 3          -                0.0   
               Governance - Season 2          -                0.0   
               Governance - Season 1          -                0.0   
               Governance - Phase 0           2               1.2M   
Grand Total                                  70              54.6M   

                                     % OP Allocated  
Status         Source                                
Live ‎🔥        Sub Total                        76%  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Coming soon ‎⏳ Sub Total                        21%  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Completed      Sub Total                         2%  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Grand Total                                       -

In [ ]:
# display new programs in last 30 days
df_new_programs = df_info[df_info["announced_date"]>pd.Timestamp("today") - timedelta(days = LAST_N_DAYS)].sort_values(by="announced_date", ascending=False)
if not df_new_programs.empty:
    df_new_programs["End Date"].fillna("-", inplace=True)
    display(df_new_programs)

In [ ]:
# display completed programs in last 30 days
df_completed = df_info[(df_info["Status"] == "Completed") & (df_info["End Date"]>pd.Timestamp("today") - timedelta(days = LAST_N_DAYS))]\
        .sort_values(by="announced_date", ascending=False)
if not df_completed.empty:
        display(df_completed)

# Usage and TVL Attribution
To combine all sources of data together

### `DF_INFO` Cleanup

In [ ]:
# create app_name_join, coalesce with app name override map, app name and remove any space
df_info["app_name_join"] = df_info["App Name Map Override"].fillna(df_info["App Name"])
df_info["app_name_join"] = df_info["app_name_join"].apply(cleanup_string)

# apply the function to the "Source" column and create a new column "Source Extracted"
df_info["source_cleanup"] = df_info["Source"].apply(extract_source)
df_info["source_cleanup"] = df_info["source_cleanup"].apply(cleanup_string)

### `DF_USAGE` Cleanup

In [ ]:
df_usage = pd.read_csv("csv_outputs/" + "dune_op_program_performance_summary" + ".csv")
df_usage["app_name_join"] = df_usage["app_name"].apply(cleanup_string)
df_usage["source_cleanup"] = df_usage["op_source"].apply(cleanup_string)

### `DF_TVL` Cleanup

In [ ]:
df_tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
df_tvl = df_tvl[df_tvl["include_in_summary"] == 1]

# cleanup
df_tvl["app_name_join"] = df_tvl["app_name"].apply(cleanup_string)
df_tvl["source_cleanup"] = df_tvl["op_source"].apply(extract_source)
df_tvl["source_cleanup"] = df_tvl["source_cleanup"].apply(cleanup_string)

df_tvl = pd.DataFrame(df_tvl.groupby(["app_name_join", "source_cleanup"]).sum()["cumul_last_price_net_dollar_flow"]).reset_index()
df_tvl

### `DF_OP_DISTRIBUTION` Cleanup

In [ ]:
df_op_distribution = pd.read_csv("csv_outputs/dune_op_distribution_type.csv")
df_op_distribution.head()


In [ ]:
result = df_op_distribution.groupby("from_name").sum().sort_values(by="op_deployed", ascending=False).reset_index()



In [ ]:
result["from_name"] = result["from_name"].apply(cleanup_string)
result[["from_name", "op_deployed"]]

### Combine Data

In [ ]:
df_combined = pd.merge(df_info, df_usage, on=["app_name_join", "source_cleanup"])
df_combined = df_combined[COL_NAMES_TO_INCLUDE]

In [ ]:
df_combined

In [ ]:
agg_dict = {
    "incremental_addr_per_day": "sum",
    "incremental_txs_per_day": "sum",
    "incremental_gas_fee_eth_per_day": "sum"
}

In [ ]:
result_fund_source = df_combined.groupby("Source").agg(agg_dict)
result_fund_source.sort_values("Source")

In [ ]:
result_fund_source = df_combined.groupby("category").agg(agg_dict)
result_fund_source.sort_values("incremental_txs_per_day", ascending=False)

In [ ]:
# program_df_grouped = program_df.groupby('App Name Map').agg({
#     '# OP Allocated': 'sum',
#     'Source': lambda x: list(set(x))
# }).reset_index()

In [ ]:
# summary by app, create array.list for season they belong to 
# have two summary, one just gov fund, the other one include partnerfund 
# use no sybil filters for this